In [ ]:
#@title Imports
import os
import re


# SimpleTokenzierV1

In [ ]:
# for item in os.listdir('.'): # '.' refers to the current directory
#     print(item)

os.getcwd()

In [ ]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_txt = f.read()

print(len(raw_txt))


In [ ]:
# Remove whitespace or not?
#   Removing whitespaces reduces memory and computing requirement
#   White spaces can be useful for text sensitive to the structure, like python indention
preprossed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_txt)
preprossed = [t.strip() for t in preprossed if t.strip()]
print(preprossed[:10])
print(len(preprossed))

In [ ]:
# Vocabulary: all the unique tokens in alphbetically order


# For tokens not in the vocab
UNKNOWN_TOKEN = "<|unk|>"

# Added between text sources.
# Allow the LLM to process and understand the data better.
END_OF_TEXT_TOKEN = "<|endoftext|>"

# Following special tokens are used by different types of tokenizers
# [BOS]: beginning of sequence
# [EOS]: end of sequence
# [PAD]: padding

sorted_unique_tokens = sorted(set(preprossed))
sorted_unique_tokens.extend([END_OF_TEXT_TOKEN, UNKNOWN_TOKEN])
print(len(sorted_unique_tokens))

In [ ]:
# Encode token to token id
vocab = {token:id for id,token in enumerate(sorted_unique_tokens)}

# for i, item in enumerate(vocab.items()):
#   print(i, item)
#   if i > 20:
#     break

In [ ]:
class SimpleTokenzierV1:
  def __init__(self, vocab):
    self.token_to_id = vocab
    self.id_to_token = {id:token for token,id in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [t.strip() for t in preprocessed if t.strip()]
    preprocessed = [t if t in self.token_to_id else UNKNOWN_TOKEN for t in preprocessed]
    return [self.token_to_id[t] for t in preprocessed]

  def decode(self, ids):
    tokens = [self.id_to_token[id] for id in ids]
    text = " ".join(tokens)
    return re.sub(r'\s+([,.?!"()\'])', r'\1', text)

In [ ]:
tokenizer_v1 = SimpleTokenzierV1(vocab)

In [ ]:
text1 = "how are you, jessica!"
text2 = "do you like tea?"
test_ids = tokenizer_v1.encode(" <|endoftext|> ".join((text1, text2)))
print(test_ids)
print(tokenizer_v1.decode(test_ids))

# BPE Tokenizer

In [ ]:
import tiktoken


# The immediate space preceding the word and the word itself are encoded as a single token

END_OF_TEXT_TOKEN = "<|endoftext|>"

tokenizer_bpe = tiktoken.get_encoding('gpt2') # download pre-trained vocabulary and merge rules

# texts = ["", "I'm", "I'm"]
# test_ids = tokenizer_bpe.encode(END_OF_TEXT_TOKEN.join(texts), allowed_special={END_OF_TEXT_TOKEN})
# print(test_ids)
# print(tokenizer_bpe.decode(test_ids))

# Dataset and DataLoader


In [ ]:
from torch.utils.data import Dataset, DataLoader

class DatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

    for i in range(0, len(token_ids) - max_length, stride):
      # One pair contains max_length training targets
      self.input_ids.append(torch.tensor(token_ids[i:i+max_length]))
      self.target_ids.append(torch.tensor(token_ids[i+1:i+1+max_length]))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

# Dataload will load the dataset efficiently
# batch_size: The data the model has to process before updating the parameters
#             The number of tensor pairs each dataloader iteration return
#             Smaller batch_size requires less memory but more noisy small updates.
#             Larger batch_size will make less noisy updates but take more time.
# max_length: The context length (the sliding window size)
# drop_last:  To drop the last batch if it's shorter to prevent loss spike during training
# stride: word overlapping will create overfitting, larger stride also help go through the text faster
# num_workders: process the input in parallel
def create_dataloader_v1(txt, batch_size, max_length, stride, shuffle=False, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = DatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
  return dataloader


In [ ]:
# dataloader = create_dataloader_v1(raw_txt)
# data_iter = iter(dataloader)
# first_batch = next(data_iter)
# print(first_batch)

# Token Embeddings and Position Embeddings

In [ ]:
# import gensim.downloader as api

# # 300 dimension
# # huggingface.co/fse/word2vec-google-news-300
# # Download the vector
# word_vectors = api.load("word2vec-google-news-300")


In [ ]:
# print(word_vectors['computer'])
# print(word_vectors.most_similar(positive=['king', 'woman'], negative=['man'], topn=10))
# print(word_vectors.similarity(['woman', 'man']))
# print(word_vectors.similarity(['tokyo', 'kyoto']))
# print(word_vectors.similarity(['fish', 'bicycle']))
# print(np.linalg.norm(word_vectors['women'] - word_vectors['man']))
# print(np.linalg.norm(word_vectors['snow'] - word_vectors['pixel']))

In [ ]:
import torch

# Create a embedding layer weight matrix
vocab_size = 50000
embedding_size = 8
context_length = 4
batch_size = 2
d_out = 15
num_heads = 3

torch.manual_seed(123)

# A simple lookup table that stores embedding of a fixed dictionary and size.
# Initialized to random numbers.
embedding_layer = torch.nn.Embedding(vocab_size, embedding_size)

# Position embedding weight matrix
pos_embedding_layer = torch.nn.Embedding(context_length, embedding_size)


# print(embedding_layer.weight)
# print(embedding_layer(torch.tensor([3])))

# input_ids = torch.tensor([2, 3, 5, 1])
# print(embedding_layer(input_ids))

In [ ]:
import tiktoken

with open('the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_txt = f.read()

print(len(raw_txt))

dataloader = create_dataloader_v1(raw_txt, batch_size=batch_size, max_length=context_length, stride=context_length)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)


token_embeddings = embedding_layer(inputs) # batch_size x context_length x embedding_size
# print(token_embeddings.shape)
# print(input)
# print(torch.arange(0, context_length))
pos_embeddings = pos_embedding_layer(torch.arange(0, context_length)) # context_length x embedding_size
input_embeddings = token_embeddings + pos_embeddings # python broadcasting
print(input_embeddings)

# Self Attention

In [ ]:

class SelfAttentionV1(torch.nn.Module):
  def __init__(self, embedding_size, d_out):
    super().__init__()
    self.Wq = torch.nn.Parameter(torch.rand(embedding_size, d_out), requires_grad=False)
    self.Wk = torch.nn.Parameter(torch.rand(embedding_size, d_out), requires_grad=False)
    self.Wv = torch.nn.Parameter(torch.rand(embedding_size, d_out), requires_grad=False)

  def forward(input_embeddings):
    Q = input_embeddings @ self.Wq
    K = input_embeddings @ self.Wk
    V = input_embeddings @ self.Wv

    print(Q.shape, K.shape, V.shape)

    attention_scores = Q @ K.transpose(-1, -2)
    attention_weights = torch.softmax(attention_scores / d_out**0.5, dim = -1)
    context_vectors = attention_weights @ V

    print(context_vectors[0])
    return context_vectors


In [ ]:

class SelfAttentionV2(torch.nn.Module):
  def __init__(self, embedding_size, d_out, qkv_bias=False):
    super().__init__()
    self.Wq = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    self.Wk = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    self.Wv = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)

  def forward(input_embeddings):
    Q = self.Wq(input_embeddings)
    K = self.Wk(input_embeddings)
    V = self.Wv(input_embeddings)

    print(Q.shape, K.shape, V.shape)

    attention_scores = Q @ K.transpose(-1, -2)
    attention_weights = torch.softmax(attention_scores / d_out**0.5, dim = -1)
    context_vectors = attention_weights @ V

    print(context_vectors[0])
    return context_vectors


In [ ]:
class CausalAttentionV1(torch.nn.Module):
  def __init__(self, embedding_size, d_out, context_length, drop_out=0.2, qkv_bias=False):
    super().__init__()
    self.Wq = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    self.Wk = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    self.Wv = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    # dropout rate = 0.2
    # * 20% of the elements in the matrix is switched to 0
    # * the remaining values are scaled by 120%
    self.dropout = torch.Dropout(drop_out)
    # mask
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagnol=1))

  def forward(input_embeddings):
    batch_size, context_length, embedding_size = input_embeddings.shape
    d_out = self.Wq.shape[1]

    Q = self.Wq(input_embeddings)
    K = self.Wk(input_embeddings)
    V = self.Wv(input_embeddings)

    print(Q.shape, K.shape, V.shape)

    attention_scores = Q @ K.transpose(1, 2)

    # mask
    # context_length = input_embeddings.shape[1]
    # mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
    # masked = attention_scores.masked_fill(mask.bool(), -torch.inf)
    # _ ops are in-place
    # when the current batch has smaller context length than the mask's
    attention_scores.masked_fill_(self.mask.bool()[:context_length, :context_length], -torch.inf)

    # scale and softmax
    attention_weights = torch.softmax(attention_scores / d_out**0.5, dim = -1)

    # dropout
    # Prevents over-fitting and improves generalization performance
    attention_weights = self.dropout(attention_weights)

    # Data leakage: the weight is influenced by the masked attention weights during softmax
    # triangular lower function
    # self.mask = torch.tril(torch.ones(context_length, context_length))
    # masked_attention_weights = attention_weights * self.mask
    # row_sums = masked_attention_weights.sum(dim=1, keepdim=True)
    # masked_attention_norm = masked_attention_weights / row_sums

    context_vectors = attention_weights @ V

    print(context_vectors[0])
    return context_vectors

In [ ]:
ca = CasualAttentionV1(embedding_size, qkv_size, context_length, 0.2)
context_vectors = ca(input_embeddings)

In [ ]:
# Multi Head attention

# output is context length * num_heads
class MultiHeadAttentionWrapper(torch.nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    self.heads = torch.nn.ModuleList([CausalAttentionV1(d_in, d_out, context_length, dropout, qkv_bias)
                                for _ in range(num_heads)])

  def forward(self, input_embedding):
    # TODO: need parallization using weight splits
    return torch.cat([head(input_embedding) for head in self.heads], dim=-1)

In [ ]:
multi_head_attention = MultiHeadAttentionWrapper(embedding_size, qkv_size, context_length, 0.2, 4)
concat_context_vectors = multi_head_attention(input_embeddings)

print(concat_context_vectors.shape)
print(concat_context_vectors[0])

In [ ]:
#@title Multi Head Attention with Weight Split

class MultiHeadAttention(torch.nn.Module):
  def __init__(self, embedding_size, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads

    # Optimized for nueral netowrk, much better for backward propogation
    self.Wq = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    self.Wk = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)
    self.Wv = torch.nn.Linear(embedding_size, d_out, bias=qkv_bias)

    # To combine head outputs
    self.out_proj = torch.nn.Linear(d_out, d_out)

    self.dropout = torch.nn.Dropout(dropout)
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self, input_embeddings):
    batch_size, context_length, embedding_size = input_embeddings.shape

    Q = self.Wq(input_embeddings)
    K = self.Wk(input_embeddings)
    V = self.Wv(input_embeddings)

    # Weight split
    # Unroll the last dimention to include num_heads and head_dim
    # (batch_size, context_length, d_out) -> (batch_size, context_length, num_heads, head_dim)
    # 4-dimension tensor
    Q = Q.view(batch_size, context_length, self.num_heads, self.head_dim)
    K = K.view(batch_size, context_length, self.num_heads, self.head_dim)
    V = V.view(batch_size, context_length, self.num_heads, self.head_dim)
    # Group the matrics by "number of heads"
    # (batch_size, context_length, num_heads, head_dim) ->
    # (batch_size, num_heads, context_length, head_dim)
    Q = Q.transpose(1, 2)
    K = K.transpose(1, 2)
    V = V.transpose(1, 2)

    attention_scores = Q @ K.transpose(2, 3)

    # mask, for causal attention
    attention_scores.masked_fill_(
        self.mask.bool()[:context_length, :context_length], -torch.inf
    )

    # scale and softmax
    attention_weights = torch.softmax(attention_scores / self.head_dim**0.5, dim=-1)

    # drop out, to avoid overfitting
    attention_weights = self.dropout(attention_weights)

    # (batch_size, head_size, context_length, head_dim)
    context_vectors = attention_weights @ V

    context_vectors = context_vectors.transpose(1, 2)
    # (batch_size, context_length, d_out)
    # contiguous() to make sure they're in same memory block
    context_vectors = context_vectors.contiguous().view(batch_size, context_length, self.d_out)

    print(context_vectors.shape)

    context_vectors = self.out_proj(context_vectors)
    return context_vectors



In [ ]:
mha = MultiHeadAttention(embedding_size, d_out, context_length, 0.1, num_heads)
context_vectors = mha(input_embeddings)

print(context_vectors.shape)
print(context_vectors)